In [1]:
import pandas as pd
import numpy as np
import pyodbc
import os

In [2]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PSDW-MSQLD01;'
                      'Database=AccessData;'
                      'Trusted_Connection=yes;')
access = pd.read_sql_query(
'''declare @STARTDATE DateTIME
declare @ENDDATE DateTIME
-------------------------------------------------------
set @STARTDATE = '2020-04-01'
set @ENDDATE = '2020-04-30'
--------------------------------------------------------
(SELECT lower([Description #2]) as Person
 ,case when [Description #1] like '%san%' or [Description #1] like '%East Gate%' or [Description #1] = 'Suite 450 Front Door' then 'San Diego' when [Description #1] like '%los an%' then 'Los Angeles' when [Description #1] like '%Kansas%' then 'Kansas City' when [Description #1] like '%Las v%' then 'Las Vegas' when [Description #1] like '%utah%' then 'Utah' when [Description #1] like '%aliso%' then 'Laguna Hills' when [Description #1] like '%new york%' then 'New York' else 'Other' end as Location
  ,case when [Description #1] like '%san%' or [Description #1] like '%East Gate%' or [Description #1] = 'Suite 450 Front Door' then convert(date,[Date and Time]) when [Description #1] like '%los an%' then convert(date,[Date and Time]) when [Description #1] like '%Kansas%' then convert(date,dateadd(hour, -2, [Date and Time])) when [Description #1] like '%Las v%' then convert(date, [Date and Time]) when [Description #1] like '%utah%' then convert(date,dateadd(hour, -1, [Date and Time])) when [Description #1] like '%aliso%' then convert(date, [Date and Time]) when [Description #1] like '%new york%' then convert(date,dateadd(hour, -3, [Date and Time])) else convert(date, [Date and Time]) end as [Date (PST)]   
    ,case when [Description #1] like '%san%' or [Description #1] like '%East Gate%' or [Description #1] = 'Suite 450 Front Door' then convert(datetime,[Date and Time]) when [Description #1] like '%los an%' then convert(datetime,[Date and Time]) when [Description #1] like '%Kansas%' then convert(datetime,dateadd(hour, -2, [Date and Time])) when [Description #1] like '%Las v%' then convert(datetime, [Date and Time]) when [Description #1] like '%utah%' then convert(datetime,dateadd(hour, -1, [Date and Time])) when [Description #1] like '%aliso%' then convert(datetime, [Date and Time]) when [Description #1] like '%new york%' then convert(datetime,dateadd(hour, -3, [Date and Time])) else convert(datetime, [Date and Time]) end as [DateTime (PST)]     
	  ,case when [Description #1] like '%san%' or [Description #1] like '%East Gate%' or [Description #1] = 'Suite 450 Front Door' then convert(time,[Date and Time]) when [Description #1] like '%los an%' then convert(time,[Date and Time]) when [Description #1] like '%Kansas%' then convert(time,dateadd(hour, -2, [Date and Time])) when [Description #1] like '%Las v%' then convert(time, [Date and Time]) when [Description #1] like '%utah%' then convert(time,dateadd(hour, -1, [Date and Time])) when [Description #1] like '%aliso%' then convert(time, [Date and Time]) when [Description #1] like '%new york%' then convert(time,dateadd(hour, -3, [Date and Time])) else convert(time, [Date and Time]) end as [Time (PST)]     
      FROM [AccessData].[dbo].[Intrapass]
  where [Description #2] not in ('') and [Description #2] not like '%(%)%' and [Description #2] not like '%contractor%' and [Description #2] not like '%cleaning%' and [Description #2] not like '%Axos LA%' 
  and [Description #2] not like '%examiner%' and [Description #2] not like '%visitor%' and [Description #2] not like '%facilities%' and [Description #2] not like '%guest%' and [Description #2] not like '%day pass%'
  and case when [Description #1] like '%san%' or [Description #1] like '%East Gate%' or [Description #1] = 'Suite 450 Front Door' then convert(datetime,[Date and Time]) when [Description #1] like '%los an%' then convert(datetime,[Date and Time]) when [Description #1] like '%Kansas%' then convert(datetime,dateadd(hour, -2, [Date and Time])) when [Description #1] like '%Las v%' then convert(datetime, [Date and Time]) when [Description #1] like '%utah%' then convert(datetime,dateadd(hour, -1, [Date and Time])) when [Description #1] like '%aliso%' then convert(datetime, [Date and Time]) when [Description #1] like '%new york%' then convert(datetime,dateadd(hour, -3, [Date and Time])) else convert(datetime, [Date and Time]) end >= @STARTDATE 
  and case when [Description #1] like '%san%' or [Description #1] like '%East Gate%' or [Description #1] = 'Suite 450 Front Door' then convert(datetime,[Date and Time]) when [Description #1] like '%los an%' then convert(datetime,[Date and Time]) when [Description #1] like '%Kansas%' then convert(datetime,dateadd(hour, -2, [Date and Time])) when [Description #1] like '%Las v%' then convert(datetime, [Date and Time]) when [Description #1] like '%utah%' then convert(datetime,dateadd(hour, -1, [Date and Time])) when [Description #1] like '%aliso%' then convert(datetime, [Date and Time]) when [Description #1] like '%new york%' then convert(datetime,dateadd(hour, -3, [Date and Time])) else convert(datetime, [Date and Time]) end <= @ENDDATE)

  union all 

  (SELECT  lower(concat( SUBSTRING([Person], charindex(',',[Person]) + 2, len([Person])), ' ',SUBSTRING([Person], 0, charindex(', ',[Person])))) as Person
  ,'Omaha' as Location
   ,convert(date,dateadd(hour, -2, substring(Time, 0, len(Time) - 3))) as Date
	 ,convert(datetime,dateadd(hour, -2, substring(Time, 0, len(Time) - 3))) as DateTime
	 , convert(time,dateadd(hour, -2, substring(Time, 0, len(Time) - 3))) as Time
  FROM [AccessData].[dbo].[Intelli_m]
  where concat( SUBSTRING([Person], charindex(',',[Person]) + 2, len([Person])), ' ',SUBSTRING([Person], 0, charindex(', ',[Person]))) not like '%[0-9]%'
and concat( SUBSTRING([Person], charindex(',',[Person]) + 2, len([Person])), ' ',SUBSTRING([Person], 0, charindex(', ',[Person]))) not like '%&%'
and convert(date,dateadd(hour, -2, substring(Time, 0, len(Time) - 3))) >= @STARTDATE and convert(date,dateadd(hour, -2, substring(Time, 0, len(Time) - 3))) <= @ENDDATE)

union all 

(SELECT case when [user_name] like '%@%' then lower(substring([user_name],0,CHARINDEX( '@', [user_name]))) else  lower([user_name]) end as Person
	  ,'VPN' as Location
	  , convert(date,dateadd(hour, -7, [date])) as Date
	 ,convert(datetime,dateadd(hour, -7, [date])) as DateTime
	  , convert(time,dateadd(hour, -7, [date])) as Time
  FROM [AccessData].[dbo].[Cisco_vpn]
  where [message_id] = '722033' 
and convert(date,dateadd(hour, -7, [date])) >= @STARTDATE and convert(date,dateadd(hour, -7, [date])) <= @ENDDATE)

union all

(SELECT DISTINCT lower(concat([First Name], ' ', [Last Name])) as Person ,  'San Diego' as Location, convert(date, [Date and Time]) as Date, convert(datetime, [Date and Time]) as Datetime, convert(time, [Date and Time]) as Time
  FROM [AccessData].[dbo].[AceParking]
  where  convert(date, [Date and Time]) >= @STARTDATE and convert(date, [Date and Time]) <= @ENDDATE)''', conn)


In [3]:
conn2 = pyodbc.connect('Driver={SQL Server};'
                      'Server=psdw-jhakd01;'
                      'Database=DwSupport;'
                      'Trusted_Connection=yes;')
employee = pd.read_sql_query(
'''SELECT  DISTINCT adp.[Emp Number],
			lower(concat(right(adp.[Payroll Name], len(adp.[Payroll Name]) - (1 + charindex(',', adp.[Payroll Name]))), ' ', left(adp.[Payroll Name],  (charindex(',', adp.[Payroll Name])) - 1))) as Name1,
      lower(concat(adp.[First Name], ' ' ,adp.[Last Name]))  as Name2,
	  lower(ad.Name) as Name3
	  , case when ad.DomainName = 'corclearing.com' then NULL else lower(ad.SAMAccountName) end as VPNName
	  ,concat(right(adp.[Payroll Name], len(adp.[Payroll Name]) - (1 + charindex(',', adp.[Payroll Name]))), ' ', left(adp.[Payroll Name],  (charindex(',', adp.[Payroll Name])) - 1)) as Employee
	  ,case when adp.[Reports To Name] not like '%,%' then concat(right(adp.[Payroll Name], len(adp.[Payroll Name]) - (1 + charindex(',', adp.[Payroll Name]))), ' ', left(adp.[Payroll Name],  (charindex(',', adp.[Payroll Name])) - 1)) else
	  concat(right(adp.[Reports To Name], len(adp.[Reports To Name]) - (1 + charindex(',', adp.[Reports To Name]))), ' ', left(adp.[Reports To Name],  (charindex(',', adp.[Reports To Name])) - 1)) end as [Manager]
	   ,adp.[Term Date]
	  ,case when m.EVP IS NULL then m.EmployeeName else m.EVP end as EVP
  FROM [DwSupport].[dbo].[vwADP_INFO] as adp
  left JOIN DwSupport.AD.CombinedDomainUserAccount AD 
ON COALESCE(CASE WHEN AD.EmployeeId LIKE '%Null%' THEN NULL
WHEN LEN(AD.EmployeeId) > 6 THEN NULL
ELSE CAST(AD.EmployeeId AS INT)
END
,case when isnumeric(AD.ExtensionAttribute1) = 1 then AD.ExtensionAttribute1 else NULL end
) = adp.[Emp Number]
left join DwSupport.dbo.vwManagerHierarchy  as m
on m.[Emp Number] = ADP.[Emp Number]
where adp.[Emp Number] > 0 ''', conn2)

In [4]:
os. chdir("X:\Office of the CEO\Project\Personnel Reporting & Tracking\Building Access Reports\Data Files-Monthly Access\QuarterDataHub")
historical = pd.read_csv('historicalnamefix.csv',encoding='ANSI')
del historical['Unnamed: 0']
historical['Emp Number'] = historical['Emp Number'].astype('str')
historical['Emp Number'] = historical['Emp Number'].str.rstrip("0")
historical['Emp Number'] = historical['Emp Number'].str.rstrip(".")
historical['Emp Number'] = historical['Emp Number'].str.rjust(6, '0')
historical = historical[historical['Emp Number'] != '000nan']
people = pd.DataFrame(pd.unique(access['Person']))
people.columns = ['name']
people1 = pd.merge(people, employee, how = 'inner', left_on= 'name', right_on= 'Name1')
people2 = pd.merge(people, employee, how = 'inner', left_on= 'name', right_on= 'Name2')
people3 = pd.merge(people, employee, how = 'inner', left_on= 'name', right_on= 'Name3')
people4 = pd.merge(people, employee, how = 'inner', left_on= 'name', right_on= 'VPNName')
people1 = pd.concat([people1, people2, people3, people4]).drop_duplicates() 
people2 = pd.DataFrame(pd.merge(people, people1, how = 'left', left_on= 'name', right_on= 'name'))
del people2['Name1']
del people2['Name2']
del people2['Name3']
del people2['VPNName']
people2 = people2.drop_duplicates()
people1 = pd.DataFrame(people2[people2['Emp Number'].isna()])
people1.reset_index(drop=True)
del people1['Employee']
del people1['Manager']
del people1['Term Date']
del people1['EVP']
historical.columns = people1.columns
historical['Emp Number'] = historical['Emp Number'].astype(object)
historical['name'] = historical['name'].str.lower()
del people1['Emp Number']
people1 = pd.merge(people1, historical, how = 'inner', on ='name')
people2 = pd.DataFrame(people2[people2['Emp Number'].notna()])
people1['Emp Number'] = people1['Emp Number'].astype('str')
employee['Emp Number'] = employee['Emp Number'].astype('str')
people1['Emp Number'] = people1['Emp Number'].str.rstrip("0")
people1['Emp Number'] = people1['Emp Number'].str.rstrip(".")
people3 = pd.merge(people1, employee, how = 'inner', on ='Emp Number')
del people3['Name1']
del people3['Name2']
del people3['Name3']
del people3['VPNName']
people2 = pd.concat([people2, people3]).drop_duplicates()  
people1 = pd.merge(people, people2, how = 'left', on= 'name')
people1 = pd.merge(people, people2, how = 'left', on= 'name')
people1 = pd.DataFrame(people1[people1['Emp Number'].isna()])
del people1['Employee']
del people1['Manager']
del people1['Term Date']
del people1['EVP']
people1.to_csv('manualnamefix.csv')
employee.to_csv('nameref.csv')

###go to X:\Office of the CEO\Project\Personnel Reporting & Tracking\Building Access Reports\Data Files-Monthly Access\QuarterDataHub and update manualnamefix.csv

In [5]:
manual = pd.read_csv('manualnamefix.csv', encoding='ANSI')
del manual['Unnamed: 0']
manual['Emp Number'] = manual['Emp Number'].astype('str')
manual['Emp Number'] = manual['Emp Number'].str.rstrip("0")
manual['Emp Number'] = manual['Emp Number'].str.rstrip(".")
manual['Emp Number'] = manual['Emp Number'].str.rjust(6, '0')
manual = manual[manual['Emp Number'] != '000nan']
historical = pd.concat([historical, manual]).drop_duplicates() 
historical.to_csv('historicalnamefix.csv')
people3 = pd.merge(manual, employee, how = 'inner', on ='Emp Number')
del people3['Name1']
del people3['Name2']
del people3['Name3']
del people3['VPNName']
people2 = pd.concat([people2, people3]).drop_duplicates()  
access = pd.merge(access, people2, how = 'inner', left_on ='Person', right_on ='name')
firstscan = access[access['Location'] != 'VPN'].groupby(['Emp Number', 'Date (PST)'])['Time (PST)'].agg(min)
lastscan = access[access['Location'] != 'VPN'].groupby(['Emp Number', 'Date (PST)'])['Time (PST)'].agg(max)
firstvpn = access[access['Location'] == 'VPN'].groupby(['Emp Number', 'Date (PST)'])['Time (PST)'].agg(min)
lastvpn = access[access['Location'] == 'VPN'].groupby(['Emp Number', 'Date (PST)'])['Time (PST)'].agg(max)
final = pd.merge(firstscan, lastscan, how = 'outer', on= ['Emp Number', 'Date (PST)'])
final = pd.merge(final, firstvpn, how = 'outer', on= ['Emp Number', 'Date (PST)'])
final = pd.merge(final, lastvpn, how = 'outer', on= ['Emp Number', 'Date (PST)'])
final.columns = ['First Scan PST', 'Last Scan PST', 'First VPN PST', 'Last VPN PST']
final = final.reset_index()
firstmode = lambda x: x.value_counts().index[0]
location = access[access['Location'] != 'VPN'].groupby('Emp Number')['Location'].agg(firstmode)
final = pd.merge(final, location, how = 'left', on= ['Emp Number'])
final['Location'].fillna('Remote', inplace = True)
final['VPNAfterWork'] = 0
final.loc[final['Last Scan PST'] < final['Last VPN PST'], 'VPNAfterWork'] = 1
final['VPNBeforeWork'] = 0
final.loc[final['First Scan PST'] > final['First VPN PST'], 'VPNBeforeWork'] = 1
final['VPNNoScan'] = 0
final.loc[final['First Scan PST'].isna(), 'VPNNoScan'] = 1
final['HoursWorked'] = final['Last Scan PST'].apply(pd.to_datetime) - final['First Scan PST'].apply(pd.to_datetime)
final['HoursWorked'] = final['HoursWorked'].astype('timedelta64[s]')/ (60*60)
final = pd.merge(final, employee, how = 'left', on= ['Emp Number'])
final = final[final['Term Date'].isna()]
final.columns = ['EID', 'Date', 'EarliestScan', 'LatestScan', 'EarliestVPNScan', 'LatestVPNScan', 'Location', 'VPNAfterWork', 'VPNBeforeWork', 'VPNNoScan', 'HoursWorked', 'n1', 'n2', 'n3', 'n4', 'Payroll Name', 'Reports To Name', 'td', 'EVP']
os. chdir("X:\Office of the CEO\Project\Personnel Reporting & Tracking\Building Access Reports\Data Files-Monthly Access\QuarterDataHub\EVPReports")
final.to_csv('datamaster.csv')